## 자물쇠와 열쇠
https://school.programmers.co.kr/learn/courses/30/lessons/60059

In [ ]:
def rotate(arr, d):
    n = len(arr)
    result = [[0] * n for _ in range(n)]

    if d % 4 == 1:
        for r in range(n):
            for c in range(n):
                result[c][n - r - 1] = arr[r][c]

    elif d % 4 == 2:
        for r in range(n):
            for c in range(n):
                result[n - r - 1][n - c - 1] = arr[r][c]

    elif d % 4 == 3:
        for r in range(n):
            for c in range(n):
                result[n - c - 1][r] = arr[r][c]
    else:
        for r in range(n):
            for c in range(n):
                result[r][c] = arr[r][c]
    return result

def check(lock):
    n = len(lock) // 3
    for i in range(n, n*2):
        for j in range(n, n*2):
            if lock[i][j] != 1:
                return False
    return True


def solution(key, lock):
    m = len(key)
    n = len(lock)
    new_lock = [[0] * (n*3) for _ in range(n*3)]

    for i in range(n):
        for j in range(n):
            new_lock[n+i][n+j] = lock[i][j]

    for i in range(1, n*2):
        for j in range(1, n*2):
            for d in range(4):
                r_key = rotate(key, d)
                for x in range(m):
                    for y in range(m):
                        new_lock[i + x][j + y] += r_key[x][y]
                if check(new_lock):
                    return True
                for x in range(m):
                    for y in range(m):
                        new_lock[i + x][j + y] -= r_key[x][y]
    return False


print(solution([[0, 0, 0], [1, 0, 0], [0, 1, 1]], [[1, 1, 1], [1, 1, 0], [1, 0, 1]]))